In [1]:
import numpy as np
import pandas as pd
import pickle

username='boraozaltun'
def save_obj(obj, loc ):
    with open(loc, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(loc):
    with open(loc, 'rb') as f:
        return pickle.load(f)

def reverse_dict(my_map):
    inv_map = {v: k for k, v in my_map.items()}
    return inv_map

reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')

comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')

prodcomm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/prodcomm_2_num.pickle')
nsavcomm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/nsavcomm_2_num.pickle')
demdcomm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/demdcomm_2_num.pickle')


In [2]:
root = '/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/matrices/{}.pickle'



In [3]:
C_goods = ['farm', 'food', 'coa', 'oil','gas', 'oxd']

F_goods = ['man','bus','oxd','trans','util']

In [4]:
data = {}
n = 134
g = len(C_goods)
k = len(F_goods)

data['n'] = n
data['g'] = g
data['k'] = k

In [5]:
root_to_map = '/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/56_to_11.xlsx'
data_map = pd.read_excel(root_to_map)

list_56 = data_map['56-sector'].tolist()
list_11 = data_map['11-sector'].tolist()
compress_dict = {}

for i, item in enumerate(list_11): 
    if item not in compress_dict:
        compress_dict[item] = [comm_2_num[list_56[i]]]
    else:
        compress_dict[item].append(comm_2_num[list_56[i]])

In [6]:
compress_dict

{'bus': [46, 50, 51, 52, 53, 54],
 'coa': [14],
 'farm': [6, 7, 8, 9, 11, 12],
 'food': [0, 1, 2, 3, 4, 5, 10, 13, 18, 19, 20, 21, 22, 23, 24],
 'gas': [16],
 'man': [25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  45],
 'oil': [15],
 'osg': [55],
 'oxd': [17],
 'trans': [47, 48, 49],
 'util': [42, 43, 44]}

In [7]:
# Lets get the easy one out of the way
data['E'] = load_obj(root.format('E')).reshape((n, 1))

In [8]:
# Now production:
Y = load_obj(root.format('Y_v2'))
data['Y_g'] = np.zeros((n, g))
data['Y_k'] = np.zeros((n, k))


for i, key in enumerate(C_goods):
    data['Y_g'][:, i] = Y[compress_dict[key], :].sum(axis=0)


for i, key in enumerate(F_goods):
    data['Y_k'][:, i] = Y[compress_dict[key], :].sum(axis=0)

In [9]:
data['Y_g'].shape

(134, 6)

In [10]:
# Now demand:
D = load_obj(root.format('D'))
data['D_g_old'] = np.zeros((n, g))
data['D_k_old'] = np.zeros((n, k))


for i, key in enumerate(C_goods):
    data['D_g_old'][:, i] = D[compress_dict[key], :].sum(axis=0)


for i, key in enumerate(F_goods):
    data['D_k_old'][:, i] = D[compress_dict[key], :].sum(axis=0)

In [11]:
# Now trade
X = load_obj(root.format('X_v2'))

data['X_g'] = np.zeros((n, n, g))
data['X_k'] = np.zeros((n, n, k))


for i, key in enumerate(C_goods):
    data['X_g'][:, :, i] = X[compress_dict[key], :, :].sum(axis=0).T

for i, key in enumerate(F_goods):
    data['X_k'][:, :, i] = X[compress_dict[key], :, :].sum(axis=0).T

In [12]:
# Adjust local consumption
for i in range(n):
    temp = np.zeros(g)
    for j in range(n):
        if i != j:
            temp += data['X_g'][j, i, :]
        
    data['X_g'][i, i, :] = data['Y_g'][i,:] - temp

# Re-adjust with demand figures
idx_g = np.where(data['X_g'] < 0)
print(idx_g)

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [13]:
# data['X_g'][idx_g] = data['D_g_old'][idx_g[1:]]

# # Now go back 
# for i in range(idx_g[0].shape[0]):
#     print(idx_g[1][i], idx_g[2][i])
#     data['Y_g'][idx_g[1][i], idx_g[2][i]] += data['X_g'][:, idx_g[1][i], idx_g[2][i]].sum()

In [14]:
#data['X'][data['X']<0]

In [15]:
for i in range(n):
    temp = np.zeros(k)
    for j in range(n):
        if i != j:
            temp += data['X_k'][j, i, :]
        
    data['X_k'][i, i, :] = data['Y_k'][i,:] - temp

idx_k = np.where(data['X_k'] < 0)
print(idx_k)

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [16]:
data['D_g'] = data['X_g'].sum(axis=1)
data['D_k'] = data['X_k'].sum(axis=1)

In [17]:
## Supply == Demand

print(data['Y_k'].sum(axis=1).sum(axis=0) - data['D_k'].sum(axis=0).sum())
print(data['Y_g'].sum(axis=1).sum(axis=0) - data['D_g'].sum(axis=0).sum())

2.9802322387695312e-08
1.862645149230957e-09


In [18]:
# Nice !
print(np.where((data['X_g']/data['Y_g'].reshape((1, n, g))).sum(axis=0)<.9)) # Some values below 1
print(np.where((data['X_k']/data['Y_k'].reshape((1, n, k))).sum(axis=0)<.9))


data['lambda_2_g'] = data['X_g']/data['Y_g'].reshape((1, n, g))
data['lambda_2_k'] = data['X_k']/data['Y_k'].reshape((1, n, k))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [19]:
data['lambda_2_k'] = data['lambda_2_k'] /data['lambda_2_k'].sum(axis=0).reshape((1, n, k))
data['lambda_2_g'] = data['lambda_2_g'] /data['lambda_2_g'].sum(axis=0).reshape((1, n, g))

In [20]:
# Trade fraction
data['lambda_g'] = data['X_g']/data['D_g'].reshape((n, 1, g))
data['lambda_k'] = data['X_k']/data['D_k'].reshape((n, 1, k))

In [21]:
data['lambda_k'] = data['lambda_k'] /data['lambda_k'].sum(axis=1).reshape((n, 1, k))

In [22]:
data['lambda_g'] = data['lambda_g'] /data['lambda_g'].sum(axis=1).reshape((n, 1, g))

In [23]:
# Now interlinkage
d = np.around(load_obj(root.format('d_l')), decimals=6)

data['d'] = np.zeros((n, k, g))

for i, comm in enumerate(C_goods):
    for j, final in enumerate(F_goods):
        data['d'][:, j, i] =  d[compress_dict[comm], :, :].sum(axis=0)[compress_dict[final], :].sum(axis=0)

In [24]:
np.where(data['d'].sum(axis=1) == 0)

(array([], dtype=int64), array([], dtype=int64))

In [25]:
data['d'] = data['d']/data['d'].sum(axis=1).reshape((n, 1, g))
data['d'].sum(axis=1)

array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1.

In [26]:
# Share of labor
e_labor = load_obj(root.format('e_labor'))

data['e_L'] = e_labor.reshape((n, 1))


# Share of commodities
e_comm = load_obj(root.format('e_comm'))
data['e'] = np.zeros((n, g))

for i, key in enumerate(C_goods):
    data['e'][:, i] = e_comm[compress_dict[key], :].sum(axis=0)


In [27]:
data['e_L'] =( 1 - data['e'].sum(axis=1)).reshape((n, 1))

In [28]:
# alpha
alpha = load_obj(root.format('alpha'))
data['alpha'] = np.zeros((n, k))

for i, key in enumerate(F_goods):
    data['alpha'][:, i] = alpha[compress_dict[key], :].sum(axis=0)

In [29]:
data['alpha'].shape

(134, 5)

In [30]:
# Doing this so that equation (15) works out
data['alpha'] = data['alpha']/data['alpha'].sum(axis=1).reshape(n, 1)

In [31]:
idx_k

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [32]:
idx_g

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [41]:
# phi
phi_labor = load_obj(root.format('phi_labor'))
phi_comm = load_obj(root.format('phi_comm'))
phi_natlres = load_obj(root.format('phi_natlres')) # See if you need to integrate land
phi_land = load_obj(root.format('phi_land'))

data['phi_L_g'] = np.zeros((n, g))
data['phi_L_k'] = np.zeros((n, k))

data['phi_R'] = np.zeros((n, g))

data['phi'] = np.zeros((n, k, g))

for i, key in enumerate(C_goods):
    data['phi_L_g'][:, i] = phi_labor[compress_dict[key], :].sum(axis=0)
    data['phi_R'][:, i] = phi_natlres[compress_dict[key], :].sum(axis=0) + phi_land[compress_dict[key], :].sum(axis=0)
    
for i, key in enumerate(F_goods):    
    data['phi_L_k'][:, i] = phi_labor[compress_dict[key], :].sum(axis=0)
    
for i, comm in enumerate(C_goods):
    for j, final in enumerate(F_goods):
        data['phi'][:, j, i] = phi_comm[compress_dict[comm],:,:].sum(axis=0)[compress_dict[final], :].sum(axis=0)
        

In [42]:
np.where(data['phi'].sum(axis=2) == 0)

(array([], dtype=int64), array([], dtype=int64))

In [43]:
np.where(data['phi'].sum(axis=2) == 1)

(array([], dtype=int64), array([], dtype=int64))

In [44]:
normalizer = (data['phi_L_k'] + data['phi'].sum(axis=2))
data['phi'] = data['phi']/(normalizer.reshape(n, k, 1))
data['phi_L_k'] = data['phi_L_k']/normalizer

In [50]:
(data['phi_L_k'] + data['phi'].sum(axis=2))

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1.

In [51]:
# Update based on notes:
# data['phi'] = data['d']*data['D_g'].reshape((n, 1, g))/data['Y_k'].reshape((n, k, 1))

# data['phi_L_k'] = 1 - data['phi'].sum(axis=2)

#print(np.where(data['phi'] >1),'blah', np.where(data['phi_L_k'] <=0))


In [52]:
normalizer = (data['phi_L_g'] + data['phi_R'])
data['phi_L_g'] = data['phi_L_g']/normalizer
data['phi_R'] = data['phi_R']/normalizer

In [53]:
(data['phi_L_g'] + data['phi_R'])

array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1.

In [54]:
data['theta_k'] = np.ones((k, 1))*5.
data['theta_g'] = np.ones((g, 1))*5.

data['eta'] = np.ones((k, 1))*0.4
data['rho_g'] = np.ones((g, 1))*0.4

data['sigma'] = 0.4

In [55]:
### Change non zero elements to zero ###

In [56]:
for i in data.keys():
    print(i)
    if i not in ['n', 'g','k','sigma']:
        data[i][data[i]<0] = 0

n
g
k
E
Y_g
Y_k
D_g_old
D_k_old
X_g
X_k
D_g
D_k
lambda_2_g
lambda_2_k
lambda_g
lambda_k
d
e_L
e
alpha
phi_L_g
phi_L_k
phi_R
phi
theta_k
theta_g
eta
rho_g
sigma


In [57]:
data['E_L'] = (data['phi_L_g']*data['Y_g']).sum(axis=1) + (data['phi_L_k']*data['Y_k']).sum(axis=1)

In [58]:
loc = '/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/data_subset_20200304.pickle'
save_obj(data, loc)